In [ ]:

import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2

test_csv = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv'
test_df = pd.read_csv(test_csv)
test_df['image_path'] = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test/' + test_df['image_id']

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

class SoilDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

test_loader = DataLoader(SoilDataset(test_df, val_transform), batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("best_soil_model.pth", map_location=device))
model = model.to(device)
model.eval()

test_predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, 1)
        test_predictions.extend(preds.cpu().numpy())

test_df['label'] = [str(pred) for pred in test_predictions]
submission = test_df[['image_id', 'label']]
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")
